In [2]:
import ee
import geemap
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import rasterio
import matplotlib.pyplot as plt
import zipfile
from osgeo import gdal
from osgeo import ogr
from osgeo import gdalconst
import contextlib

In [3]:
# Pre-Processing
def preProc(in_fn, out_fn, crs):
    """
        Function for preprocessing vector data. It reads the file, makes its geometry valid, and reprojects it in a desired CRS.
    """
    gdf = gpd.read_file(in_fn)
    gdf['ID'] = 1
    gdf.geometry = gdf.make_valid()
    gdf = gdf[['ID','geometry']]
    gdf = gdf.dissolve()
    
    gdf.to_crs(crs).to_file(out_fn, driver = 'GeoJSON')

In [4]:
# preProc('../Data/Vector/Cashew_Tanzania.shp', '../Data/Vector/Cashew_Tanz.geojson', 'epsg:32736') # Tanzania
preProc('../Data/Vector/Cashew_Ivorycoast.shp', '../Data/Vector/Cashew_Ivy.geojson', 'epsg:2041') # IvyCst

In [5]:
def rasterize_shp(base, shp, output, crs = None, split = None):
    """
    Function to rasterize vector data
        Inputs:
            - base: Base raster 
            - shp: Vector filename
            - output: Output geotiff filename
            - crs: coordinate reference system EPSG code
            - split: Split set to be rasterized (Can be train, validation or test)
        Outputs:

        Process:
            - Exports rasterize version of vector data
        
    """
    if split != None:
        n_shp = '../Data/Vector/temp.geojson'
        if 'temp.geojson' in os.listdir('../Data/Vector'):
            os.remove(n_shp)
        gdf = gpd.read_file(shp)
        gdf = gdf[gdf['split'] == split]
        gdf.to_file(n_shp, driver = 'GeoJSON')
        shp = n_shp
        preProc(shp, shp, crs)
        
    data = gdal.Open(base, gdalconst.GA_ReadOnly)
    geo_transform = data.GetGeoTransform()
    
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize
    
    vec = ogr.Open(shp)
    lyr = vec.GetLayer()
    pixel_width = geo_transform[1]
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
    
    band = target_ds.GetRasterBand(1)
    band.FlushCache()
    
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values = [1])
    
    target_ds = None
    
    if split != None:
        os.remove(shp)

## For Tanzania

In [6]:
dir = '../Data/Planet'

sets = ['Train', 'Validation', 'Test']

for s in sets:
    for i in os.listdir(dir):
        if 'Tanzania' + s in i:
            base = '../Data/Planet/' + i
            shp = '../Data/Vector/Cashew_Polygons_TNZ_splitted_KM.geojson'
            output = '../Data/Vector/Tanzania'+s+'Rasterized'+i.split('.')[0][-1]+'.tif'
            with contextlib.suppress(PermissionError):
                rasterize_shp(base, shp, output, 'epsg:32736', split = s.lower())

## For Ivory Coast

In [6]:
dir = '../Data/Planet'

sets = ['Train', 'Validation', 'Test']

for s in sets:
    for i in os.listdir(dir):
        if 'IvoryCoast' + s in i:
            base = '../Data/Planet/' + i
            shp = '../Data/Vector/Cashew_Polygons_CIV_splitted_KM.geojson'
            output = '../Data/Vector/IvoryCoast'+s+'Rasterized'+i.split('.')[0][-1]+'.tif'
            print(s, i)
            with contextlib.suppress(PermissionError):
                rasterize_shp(base, shp, output, 'epsg:2041', split = s.lower())

Train IvoryCoastTrainStudyArea1.tif
Train IvoryCoastTrainStudyArea10.tif
Train IvoryCoastTrainStudyArea11.tif
Train IvoryCoastTrainStudyArea12.tif
Train IvoryCoastTrainStudyArea13.tif
Train IvoryCoastTrainStudyArea14.tif
Train IvoryCoastTrainStudyArea15.tif
Train IvoryCoastTrainStudyArea2.tif
Train IvoryCoastTrainStudyArea3.tif
Train IvoryCoastTrainStudyArea4.tif
Train IvoryCoastTrainStudyArea5.tif
Train IvoryCoastTrainStudyArea6.tif
Train IvoryCoastTrainStudyArea7.tif
Train IvoryCoastTrainStudyArea8.tif
Train IvoryCoastTrainStudyArea9.tif
Validation IvoryCoastValidationStudyArea1.tif
Validation IvoryCoastValidationStudyArea2.tif
Validation IvoryCoastValidationStudyArea3.tif
Validation IvoryCoastValidationStudyArea4.tif
Validation IvoryCoastValidationStudyArea5.tif
Validation IvoryCoastValidationStudyArea6.tif
Validation IvoryCoastValidationStudyArea7.tif
Validation IvoryCoastValidationStudyArea8.tif
Test IvoryCoastTestStudyArea1.tif
Test IvoryCoastTestStudyArea1.tif.aux.xml


AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'